In [ ]:
%pip install bitshuffle
%pip install Pillow
%pip install scikit-image
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
#from ultralytics import YOLO
import time
import os
import random
import psutil
import scipy.ndimage
from PIL import Image
from PIL import ImageDraw
from pathlib import Path
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
import cv2
from scipy.ndimage import label as connected_components
from skimage.filters import sobel
from skimage.segmentation import slic
from skimage.util import img_as_float
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from random import shuffle
from tqdm import tqdm
from skimage.util import img_as_float
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from skimage.segmentation import slic
%matplotlib inline

In [ ]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
36553,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36554,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36555,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36556,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df['.h5 path'] = df['.h5 path'].str.replace('0000.h5', '0002.h5', regex=False)
df = df.drop_duplicates(subset='.h5 path', keep='first').reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
30309,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30310,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30311,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30312,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df = df[~df['.h5 path'].str.contains('spliced')].reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
1,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
2,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
3,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
4,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
...,...,...,...,...,...,...,...,...
29341,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29342,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29343,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29344,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df = df.drop(index = 17546)
#df = df[df['Band'] == 'L']
# df = df.sample(n=1000, random_state=42).reset_index(drop=True)

In [ ]:
import pandas as pd
import blimpy as bl
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

def scan_file_for_band(
    h5_path: str,
    band: str,
    fmin: float,
    fmax: float,
    default_nfpc: int = 1024
):
    """
    Open one .h5, read header, return list of dicts for all channels
    whose freq range lies in [fmin, fmax].
    """
    records = []
    fb = bl.Waterfall(h5_path, load_data=False)
    hdr = fb.header

    # skip if not the right band
    # you could also pass row['Band'] in here if you like
    # if row['Band'] != band: return []

    fch1   = hdr['fch1']
    foff   = hdr['foff']
    nchans = hdr.get('nchans')
    nfpc   = hdr.get('nfpc', default_nfpc)
    n_coarse = nchans // nfpc

    for ch in range(n_coarse):
        f0 = ch * nfpc
        f1 = (ch+1) * nfpc
        f_start = fch1 + f0 * foff
        f_stop  = fch1 + (f1-1) * foff
        if f_start >= fmin or f_stop <= fmax:
            records.append({
                '.h5 path': h5_path,
                'channel':    ch,
                'Band':       band,
                'f_start':    f_start,
                'f_stop':     f_stop
            })

    return records

def parallel_filter_df(
    df: pd.DataFrame,
    band: str = "L",
    fmin: float = 1500,
    fmax: float = 1650,
    max_workers: int = 8
) -> pd.DataFrame:
    # restrict to Band==L first to cut the task list down
    df_band = df[df['Band'] == band]
    paths   = df_band[".h5 path"].unique().tolist()

    all_records = []
    with ProcessPoolExecutor(max_workers=max_workers) as exe:
        futures = {
            exe.submit(scan_file_for_band, p, band, fmin, fmax): p
            for p in paths
        }
        for fut in tqdm(as_completed(futures), total=len(futures),
                        desc=f"Scanning {band}-band files"):
            try:
                recs = fut.result()
                all_records.extend(recs)
            except Exception as e:
                h5 = futures[fut]
                print(f"Error on {h5}: {e!r}")

    return pd.DataFrame.from_records(all_records)

# Usage:
df = parallel_filter_df(df, band="L", fmin=1500, fmax=1650, max_workers=6)
print(f"Kept {len(df)} channel entries in L-band 1500-1650 MHz")
df = df.sample(n=1000, random_state=42).reset_index(drop=True)



In [ ]:
fb = bl.Waterfall(df['.h5 path'].iloc[-1], load_data=True)
fb.info()


--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :               10501.46484375 MHz
            foff :         -0.00286102294921875 MHz
           ibeam :                               -1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                            65536
            nifs :                                1
     rawdatafile : guppi_59411_54386_HIP30264_0094.0000.raw
     source_name :                         HIP30264
         src_dej :                     -8:26:53.228
         src_raj :                      6:21:58.451
    telescope_id :                                6
           tsamp :                1.073741823999999
   tstart (ISOT) :          2021-07-16T15:06:26.000
    tstart (MJD) :                59

In [ ]:
import psutil
import time

def ensure_cpu_mem(bytes_needed, safety=0.8):
    avail = psutil.virtual_memory().available
    if bytes_needed > avail * safety:
        time.sleep(120)

def ensure_gpu_mem(bytes_needed, safety=0.8):
    free, total = cp.cuda.runtime.memGetInfo()
    if bytes_needed > free * safety:
        time.sleep(120)


In [ ]:
import numpy as np

def fuse(box_lists, iou_thresh=0.3, min_width=3, max_width=30):
    """
    box_lists: list of lists of (l, r, meta) tuples from each expert
    iou_thresh: IoU threshold above which two boxes get merged
    returns: fused list of (l, r, merged_meta)
    """
    # flatten
    boxes = [b for sub in box_lists for b in sub]
    # sort by descending width or snr to merge biggest ones first
    boxes.sort(key=lambda b: (b[1]-b[0]), reverse=True)

    fused = []
    for (l, r, meta) in boxes:
        w = r-l
        if w < min_width or w > max_width:
            continue
        # try to merge with an existing fused box
        for idx,(fl,fr,fmeta) in enumerate(fused):
            # IoU = intersection / union
            inter = max(0, min(r,fr)-max(l,fl))
            union = (r-l)+(fr-fl)-inter
            if union>0 and (inter/union)>=iou_thresh:
                # merge intervals
                nl, nr = min(l,fl), max(r,fr)
                # you can also merge metadata however you like:
                fused[idx] = (nl, nr, {**fmeta, **meta})
                break
        else:
            # no overlap → keep as new box
            fused.append((l, r, meta))

    return fused


In [ ]:
import numpy as np

def iou(box1, box2):
    # box = (xc, yc, w, h) in normalized coords
    x10 = box1[0] - box1[2]/2; x11 = box1[0] + box1[2]/2
    y10 = box1[1] - box1[3]/2; y11 = box1[1] + box1[3]/2
    x20 = box2[0] - box2[2]/2; x21 = box2[0] + box2[2]/2
    y20 = box2[1] - box2[3]/2; y21 = box2[1] + box2[3]/2

    xi0 = max(x10, x20);  yi0 = max(y10, y20)
    xi1 = min(x11, x21);  yi1 = min(y11, y21)
    inter_w = max(0, xi1 - xi0)
    inter_h = max(0, yi1 - yi0)
    inter = inter_w * inter_h

    area1 = box1[2] * box1[3]
    area2 = box2[2] * box2[3]
    union = area1 + area2 - inter
    return inter/union if union>0 else 0

def non_max_suppression(boxes, scores=None, iou_thresh=0.3):
    """
    boxes: list of (xc, yc, w, h)
    scores: optional list of same length; if None we sort by box area (bigger first)
    iou_thresh: IoU above which we suppress duplicates
    """
    if len(boxes) == 0:
        return []
    if scores is None:
        scores = [w*h for (_,_,w,h) in boxes]

    # sort indices by descending score
    idxs = np.argsort(scores)[::-1]
    keep = []
    for i in idxs:
        b = boxes[i]
        if all(iou(b, boxes[j]) < iou_thresh for j in keep):
            keep.append(i)
    return [boxes[i] for i in keep]


In [ ]:
def remove_nested_boxes(boxes, scores):
    """
    Drop any box that fully contains another, choosing the higher‐score box
    when deciding which to keep. Returns (boxes_kept, scores_kept).
    """
    def to_corners(b):
        xc, yc, w, h = b
        x0, x1 = xc - w/2, xc + w/2
        y0, y1 = yc - h/2, yc + h/2
        return x0, y0, x1, y1

    corners = [to_corners(b) for b in boxes]
    keep = [True]*len(boxes)

    for i in range(len(boxes)):
        if not keep[i]:
            continue
        x0_i,y0_i,x1_i,y1_i = corners[i]
        for j in range(len(boxes)):
            if i==j or not keep[j]:
                continue
            x0_j,y0_j,x1_j,y1_j = corners[j]
            # if box i fully contains box j
            if x0_i <= x0_j and x1_i >= x1_j and y0_i <= y0_j and y1_i >= y1_j:
                # keep only the higher‐scoring one
                if scores[i] >= scores[j]:
                    keep[j] = False
                else:
                    keep[i] = False

    boxes_kept  = [b for b,k in zip(boxes, keep) if k]
    scores_kept = [s for s,k in zip(scores,keep) if k]
    return boxes_kept, scores_kept




In [ ]:
def generate_yolo_boxes(edge_map, threshold=0.1):
    binary = (edge_map > threshold).astype(np.uint8)
    labeled, n = connected_components(binary)
    boxes = []
    for i in range(1, n + 1):
        coords = np.argwhere(labeled == i)
        if coords.shape[0] < 10:
            continue
        y0, x0 = coords.min(axis=0)
        y1, x1 = coords.max(axis=0)
        cx = (x0 + x1) / 2 / edge_map.shape[1]
        cy = (y0 + y1) / 2 / edge_map.shape[0]
        w = (x1 - x0) / edge_map.shape[1]
        h = (y1 - y0) / edge_map.shape[0]
        boxes.append((cx, cy, w, h))
    return boxes

In [ ]:
def build_tasks(df):
    """
    Build a flat list of (h5_path, channel_idx) from your DataFrame,
    skipping any channels that fall into known notch filter frequency ranges.
    """
    GBT_NOTCH_FILTERS = {
        "L": [(1200, 1340)],
        "S": [(2300, 2360)],
    }

    tasks = []

    for _, row in df.iterrows():
        h5 = row[".h5 path"]
        band = row["Band"]  # e.g., 'L', 'S', etc.
        fb = bl.Waterfall(h5, load_data=False)
        nfreq = fb.header.get("nchans")
        nfpc = fb.header.get("nfpc", 1024)
        fch1 = fb.header["fch1"]
        foff = fb.header["foff"]
        n_coarse = nfreq // nfpc

        for ch in range(n_coarse):
            f0 = fch1 + ch * nfpc * foff
            f1 = fch1 + (ch + 1) * nfpc * foff
            f_min, f_max = sorted([f0, f1])

            # Check against notch filter exclusion ranges
            skip = False
            if band in GBT_NOTCH_FILTERS:
                for lo, hi in GBT_NOTCH_FILTERS[band]:
                    if lo <= f_min <= hi or lo <= f_max <= hi:
                        skip = True
                        break
            if not skip:
                tasks.append((h5, ch))

    return tasks


def split_tasks(tasks, train_frac=0.8, seed=42):
    """
    Shuffle & split the flat task list into train vs. val sets.
    Returns two sets of (h5_path, channel_idx).
    """
    random.seed(seed)
    shuffled = tasks.copy()
    random.shuffle(shuffled)
    cut = int(train_frac * len(shuffled))
    train = set(shuffled[:cut])
    val   = set(shuffled[cut:])
    return train, val

In [ ]:
from scipy.stats import kurtosis

def detect_boxes_with_patch_kurtosis(
    block,               # raw 2D power array (ntime × nfreq)
    fused,               # your fused MoE map, same shape
    threshold,           # global threshold you already compute
    kurt_thresh=3.5,     # min kurtosis inside a patch to even try detection
    tile_size=64,        # patch size in pixels
    step=None           # stride between patches (defaults to 50% overlap)
):
    H, W = block.shape
    if step is None:
        step = tile_size // 2

    all_boxes = []
    for y0 in range(0, H, step):
        for x0 in range(0, W, step):
            y1 = min(H, y0 + tile_size)
            x1 = min(W, x0 + tile_size)

            # 1) compute patch‐local kurtosis
            patch_blk = block[y0:y1, x0:x1]
            flat = patch_blk.ravel()
            if flat.size < 2:
                continue
            k_loc = kurtosis(flat, fisher=False)
            if k_loc < kurt_thresh:
                continue

            # 2) detect boxes in this patch
            patch_fused = fused[y0:y1, x0:x1]
            boxes_patch = generate_yolo_boxes(patch_fused, threshold=threshold)

            # 3) convert each box to global normalized coords
            ph, pw = y1 - y0, x1 - x0
            for cx_p, cy_p, w_p, h_p in boxes_patch:
                # patch‐pixel center
                xc_pix = cx_p * pw  
                yc_pix = cy_p * ph
                # convert to absolute pixel coords, then normalize
                xc = (x0 + xc_pix) / W
                yc = (y0 + yc_pix) / H
                w  = (w_p * pw) / W
                h  = (h_p * ph) / H
                all_boxes.append((xc, yc, w, h))

    return all_boxes


In [ ]:
import numpy as np

def shrink_large_boxes(
    boxes: list[tuple[float, float, float, float]],
    fused: np.ndarray,
    w_max: float = 0.9,
    h_max: float = 0.9,
    global_thresh: float = None,
    local_frac: float = 0.1,
    min_coverage: float = 0.05
) -> list[tuple[float, float, float, float]]:
    """
    For any box whose normalized width >= w_max OR height >= h_max,
    re-extract the patch in `fused`, threshold it at `mask_thr` and
    only shrink if the mask covers at least `min_coverage` fraction
    of the patch.  Otherwise, keep the original box.

    Parameters:
    - boxes: YOLO boxes (xc, yc, w, h) normalized coords
    - fused: 2D fused map
    - w_max, h_max: thresholds for deciding "large" boxes
    - global_thresh: if provided, used as local mask threshold;
                     otherwise mask_thr = local_frac * fused.max()
    - local_frac: fraction of fused.max() used if global_thresh is None
    - min_coverage: minimum fraction of patch pixels above mask_thr
                     to allow shrinking
    """
    H, W = fused.shape
    # Determine local mask threshold
    if global_thresh is None:
        mask_thr = local_frac * fused.max()
    else:
        mask_thr = global_thresh

    shrunk = []
    for xc, yc, w, h in boxes:
        # only shrink boxes that exceed size
        if w < w_max and h < h_max:
            shrunk.append((xc, yc, w, h))
            continue

        # map normalized center/size → pixel coords
        x0 = max(0, int((xc - w/2) * W))
        x1 = min(W, int((xc + w/2) * W))
        y0 = max(0, int((yc - h/2) * H))
        y1 = min(H, int((yc + h/2) * H))

        patch = fused[y0:y1, x0:x1]
        if patch.size == 0:
            shrunk.append((xc, yc, w, h))
            continue

        # binarize and check coverage
        mask = patch > mask_thr
        cov = mask.sum() / mask.size
        if cov < min_coverage:
            # not enough core pixels: skip shrinking
            shrunk.append((xc, yc, w, h))
            continue

        ys, xs = np.nonzero(mask)
        y_min, y_max = ys.min(), ys.max()
        x_min, x_max = xs.min(), xs.max()

        # convert back to normalized coords
        new_w = (x_max - x_min + 1) / W
        new_h = (y_max - y_min + 1) / H
        new_xc = (x0 + (x_min + x_max + 1)/2) / W
        new_yc = (y0 + (y_min + y_max + 1)/2) / H

        shrunk.append((new_xc, new_yc, new_w, new_h))

    return shrunk


In [ ]:
import numpy as np
from astropy.stats import sigma_clip
from scipy.signal import find_peaks
import traceback

def clipped_2D_bounds_numpy(
    data: np.ndarray,
    min_empty_bins: int = 2,
    min_clipped:    int = 1,
    peak_prominence:int = 4
):
    """
    Run sigma-clip on a 2D intensity array (time × freq), and
    find the left/right frequency-bin bounds around the strongest
    central peak of clipped pixels.
    
    Parameters
    ----------
    data : 2D np.ndarray
        Intensity data, shape (n_time, n_freq).
    min_empty_bins : int
        Kernel size for requiring adjacent non-zero runs.
    min_clipped : int
        Minimum clipped pixels along time for a freq-bin to count.
    peak_prominence : int
        Minimum prominence (in clipped-pixel counts) for peak finding.
    
    Returns
    -------
    l : int
        Left frequency-bin index (inclusive).
    r : int
        Right frequency-bin index (exclusive).
    metadata : dict
        { 'num_peaks': <int>, 'peaks': <peaks tuple from scipy> }
    """
    # 1) Normalize to [0,1]
    arr = data.astype(float)
    arr -= arr.min()
    mx = arr.max()
    if mx > 0:
        arr /= mx

    # 2) Sigma-clip along time axis
    clipped = sigma_clip(arr, axis=0)
    mask_spec = np.sum(clipped.mask, axis=0)  # counts of clipped pixels per freq

    # 3) Peak finding in that “clipped count” spectrum
    peaks = find_peaks(mask_spec, prominence=peak_prominence)
    if len(peaks[0]) == 0:
        raise ValueError("No peaks found in clipped-count spectrum")

    # choose the peak of highest prominence, nearest the center
    center = mask_spec.size // 2
    prominences = peaks[1]['prominences']
    max_idxs    = np.where(prominences == prominences.max())[0]
    peak_i      = peaks[0][ max_idxs[np.argmin(np.abs(peaks[0][max_idxs] - center))] ]

    # 4) Threshold mask and convolve to enforce runs of non-zero
    good = (mask_spec >= min_clipped).astype(int)
    conv = np.convolve(good, np.ones(min_empty_bins, dtype=int), mode='valid')
    c_mask = (conv != 0).astype(int)
    diffs  = np.diff(c_mask)

    # 5) Rising / falling edges around peak_i
    l_idx = np.where(diffs > 0)[0]
    r_idx = np.where(diffs < 0)[0]

    # pick last rising edge to the left, first falling to the right
    l = l_idx[l_idx + min_empty_bins <= peak_i][-1] + min_empty_bins
    r = r_idx[r_idx >= peak_i][0] + 1

    metadata = {
        'num_peaks': len(peaks[0]),
        'peaks':     peaks
    }
    return l, r, metadata


In [ ]:
import numpy as np
from astropy.stats import sigma_clip

def threshold_baseline_bounds_numpy(spec: np.ndarray, p: float = 0.01):
    """
    Pure-NumPy version of threshold_baseline_bounds.

    Parameters
    ----------
    spec : 1D np.ndarray
        Input intensity spectrum.
    p : float
        Fraction of the peak (0 < p < 1) at which to set bounds.

    Returns
    -------
    l : int
        Left bin index (inclusive) of the thresholded region.
    r : int
        Right bin index (exclusive) of the thresholded region.
    metadata : dict
        {
          'noise_mean': mean of sigma-clipped noise,
          'spec_max':   maximum of baseline-subtracted spectrum
        }
    """
    # 1) Estimate noise via sigma-clip
    noise_spec = sigma_clip(spec, masked=True)
    x = np.arange(spec.size)

    # 2) Fit a 1st-degree baseline to the unmasked (i.e. noise) points
    good = ~noise_spec.mask
    coeffs = np.polyfit(x[good], noise_spec.data[good], deg=1)
    poly   = np.poly1d(coeffs)

    # 3) Subtract baseline and normalize by its maximum
    spec_sub = spec - poly(x)
    spec_max = spec_sub.max()
    if spec_max <= 0:
        raise ValueError("Spectrum has non-positive dynamic range after baseline subtraction")
    norm_spec = spec_sub / spec_max

    # 4) Find all bins below fraction p of the peak
    cutoffs = np.where(norm_spec < p)[0]
    if cutoffs.size < 2:
        raise ValueError("Not enough cutoff points found; check p or input spectrum")

    # 5) Locate the central peak in the normalized spectrum
    peak_i = int(np.argmax(norm_spec))

    # 6) Digitize to find which gap interval contains the peak
    idx = np.digitize(peak_i, cutoffs) - 1
    if idx < 0 or idx + 1 >= cutoffs.size:
        raise IndexError("Peak lies outside cutoff intervals")

    # 7) Bounds are the edges just outside the cutoff run containing the peak
    l = int(cutoffs[idx] + 1)
    r = int(cutoffs[idx + 1])

    metadata = {
        'noise_mean': float(np.mean(noise_spec.data)), 
        'spec_max':   float(spec_max)
    }
    return l, r, metadata


In [ ]:
# The MoE: Structured Forest (SF), Sobel (UED), Canny, HOG (Histogram of Oriented Gradients)

import cv2
import numpy as np
import torch
from torch import nn
from pathlib import Path
from PIL import Image, ImageDraw
import blimpy as bl
from skimage.segmentation import slic
from skimage.feature import hog
from skimage.transform import probabilistic_hough_line
from scipy.stats import kurtosis
from setigen.frame import Frame


# --- Define RuleGate  ------------------------------------------------------
class RuleGate(nn.Module):
    def __init__(self,
                 w_align=3.0,   # weight on alignment diff
                 w_ent=0.8,     # weight on entropy diff
                 w_dens=0.5,    # (optional) weight on density diff
                 w_lin=3.0,     # weight on linearity diff
                 bias=-0.2,     # bias for the mix‐gate
                 line_thresh=0.2 # baseline linearity to trigger pure Hough
                ):
        super().__init__()
        self.w_align    = w_align
        self.w_ent      = w_ent
        self.w_dens     = w_dens
        self.w_lin      = w_lin
        self.bias       = bias
        self.line_thresh = line_thresh

    def forward(self,
                h_sf: torch.Tensor,  # (B,4)
                h_ued: torch.Tensor,  # (B,4)
                h_canny: torch.Tensor,  # (B,4)
                h_hough: torch.Tensor   # (B,4)
               ):
        # ---- stage 1: Hough "confidence" gate ----
        line_diff = h_hough[:, 3] - self.line_thresh
        p_hough   = torch.sigmoid(self.w_lin * line_diff)

        # ---- stage 2: two-expert mix: E1 = avg(SF, Canny), E2 = UED ----
        # alignment difference
        e1_align   = 0.5 * (h_sf[:,1] + h_canny[:,1])
        e2_align   =       h_ued[:,1]
        align_diff = e2_align - e1_align

        # entropy difference (penalize when SF+Canny more textured)
        e1_ent  = 0.5 * (h_sf[:,2] + h_canny[:,2])
        e2_ent  =       h_ued[:,2]
        ent_diff = e1_ent - e2_ent

        # density difference
        e1_dens   = 0.5 * (h_sf[:,0] + h_canny[:,0])
        e2_dens   =       h_ued[:,0]
        dens_diff = e2_dens - e1_dens

        # mix score: favor UED when alignment and density high, penalize entropy
        score_mix = (
            self.w_align   * align_diff
          - self.w_ent     * ent_diff
          + self.w_dens    * dens_diff
          + self.bias
        )
        p_mix = torch.sigmoid(score_mix)

        return p_hough, p_mix



# -- 2) Heuristic helper -------------------------------------------------
def compute_heuristics(edge_map, gray):
    mask = edge_map > 0.2
    density = float(mask.mean())
    if density > 0:
        gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, 3)
        gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, 3)
        alignment = float(np.hypot(gx, gy)[mask].mean())
    else:
        alignment = 0.0
    entropy = float(-np.sum(edge_map * np.log2(edge_map + 1e-8)))
    bw = mask.astype(np.uint8)
    n, labels = cv2.connectedComponents(bw)
    # build ratios as you already do…
    ratios = []
    for L in range(1, n):
        ys, xs = np.where(labels == L)
        if ys.size:
            h, w = np.ptp(ys)+1, np.ptp(xs)+1
            ratios.append(w/h if h else 0.0)

    # simple list-truth test:
    if ratios:
        linearity = float(np.mean(ratios))
    else:
        linearity = 0.0

    if density < 0.01:
        return [0.0, 0.0, 0.0, 0.0]
    return [density, alignment, entropy, linearity]


# -------- 3) process_file --------------------------------------------
def process_file(job):
    (h5_path, channels, gidxs,
     base_dir, class_id, train_set, val_set,
     pad_width) = job

    fb   = bl.Waterfall(h5_path, load_data=True)
    data = 10*np.log10(fb.data.squeeze())

    EDGE_MODEL = "/home/jliang/gbt-rfi/model.yml.gz"
    sf_det     = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL)

    for ch_idx, gidx in zip(channels, gidxs):
        subset = "train" if (h5_path,ch_idx) in train_set else "val"
        img_dir = Path(base_dir)/subset/"images"
        lbl_dir = Path(base_dir)/subset/"labels"
        vis_dir = Path(base_dir)/"visualization"/subset
        for d in (img_dir,lbl_dir,vis_dir): d.mkdir(parents=True, exist_ok=True)

        cw = fb.header.get("nfpc",1024)
        f0, f1 = ch_idx*cw, (ch_idx+1)*cw
        block = data[:,f0:f1]
        low, high = int(0.15*cw), int(0.85*cw)
        block = block[:,low:high]
        h_img, w_img = block.shape

        # Remove vertical line artifact
        rows, cols = block.shape
        vert_means = block.mean(axis=0)
        center = np.argmax(vert_means)
        left_col = center - 1
        right_col = center + 1
        if left_col >= 0 and right_col < cols:
            block[:, center] = (block[:, left_col] + block[:, right_col]) / 2
        elif left_col >= 0:
            block[:, center] = block[:, left_col]
        elif right_col < cols:
            block[:, center] = block[:, right_col]


        # Structured Forest
        norm = (block - block.min())/(np.ptp(block)+1e-6)
        img3 = np.stack([norm]*3, axis=-1).astype(np.float32)
        sf_map = sf_det.detectEdges(img3).squeeze()
        gray8  = (255*norm).astype(np.uint8)

        # Sobel
        gx = cv2.Sobel(gray8, cv2.CV_32F, 1,0,3)
        gy = cv2.Sobel(gray8, cv2.CV_32F, 0,1,3)
        ued_map = np.hypot(gx, gy)
        ued_map = cv2.normalize(ued_map, None, 0,1, cv2.NORM_MINMAX)

        # Canny
        low_thresh, high_thresh = 30, 100
        gray_denoised = cv2.medianBlur(gray8, 3)
        edges = cv2.Canny(gray_denoised, low_thresh, high_thresh)
        canny_map = edges.astype(np.float32) / 255.0

        # Hough
        hough_map = np.zeros_like(canny_map, dtype=np.float32)
        lines = probabilistic_hough_line(
            (edges > 0).astype(np.uint8),
            threshold=5,
            line_length=10,
            line_gap=2
        )
        for (y0, x0), (y1, x1) in lines:
            cv2.line(hough_map, (x0, y0), (x1, y1), 1.0, 1)
        hough_map = cv2.GaussianBlur(hough_map, (3,3), 0)

        bk = float(kurtosis(gray8.flatten(), fisher=False))
        # Define the RuleGate based on kurtosis
        if bk < 15.0:
            gate = RuleGate(
            w_align    = 3.5,   # more trust in edges
            w_ent      = 0.6,   # slightly less veto power
            w_dens     = 0.5,
            w_lin      = 5.0,
            bias       = -0.1,  # less bias toward SF
            line_thresh= 0.08
            )
        elif bk < 50:
            gate = RuleGate(
                w_align    = 5.0,    # more trust in edges
                w_ent      = 0.4,    # less veto from SF entropy
                w_dens     = 0.8,    # let density help a bit
                w_lin      = 6.0,    # Hough fires more easily
                bias       = 0.1,   # slight default tilt toward edges
                line_thresh= 0.06    # only 6 % linearity needed
            )

        elif bk < 150.0:
            # relaxed mid-kurtosis gate: more permissive for edges & Hough
            gate = RuleGate(
                w_align    = 5.0,    # stronger trust in edge alignment
                w_ent      = 0.4,    # less veto from SF‐entropy
                w_dens     = 0.6,    # moderate density boost
                w_lin      = 6.0,    # let Hough fire on softer linearity
                bias       =  0.1,   # slight default tilt toward edges
                line_thresh= 0.06    # lower bar for pure Hough
            )
        elif bk < 300.0:
            # high‐kurtosis → favor edges more
            gate = RuleGate(
                w_align    = 5.0,
                w_ent      = 0.3,
                w_dens     = 0.5,
                w_lin      = 6.0,
                bias       =  0.2,
                line_thresh= 0.06
            )
        else:
            # extremely‐spiky → almost‐pure Hough & edges
            gate = RuleGate(
                w_align    = 6.0,
                w_ent      = 0.1,
                w_dens     = 1.5,
                w_lin      = 6.0,
                bias       =  0.4,
                line_thresh= 0.04
            )
        

        fused = np.zeros_like(sf_map, dtype=np.float32)
        segments = slic(img3, n_segments=100, compactness=10)
        for v in np.unique(segments):
            mask = (segments==v)
            if mask.sum()<50:
                continue
            h_sf = compute_heuristics(sf_map[mask], gray8[mask])
            h_ued = compute_heuristics(ued_map[mask], gray8[mask])
            h_canny = compute_heuristics(canny_map[mask], gray8[mask])
            h_hough = compute_heuristics(hough_map[mask], gray8[mask])
            h_sf_t = torch.tensor([h_sf], dtype=torch.float32)
            h_ued_t = torch.tensor([h_ued], dtype=torch.float32)
            h_canny_t = torch.tensor([h_canny], dtype=torch.float32)
            h_hough_t = torch.tensor([h_hough], dtype=torch.float32)
            p_line, p_mix = gate(h_sf_t, h_ued_t, h_canny_t, h_hough_t)
            p_line, p_mix = p_line.item(), p_mix.item()
            fused_region = (p_line * hough_map[mask] + (1 - p_line) * (p_mix * (ued_map[mask] + canny_map[mask]) / 2
                + (1 - p_mix) * sf_map[mask]))
            fused[mask] = fused_region

        # 1) measure & clamp kurtosis
        bk = float(kurtosis(gray8.flatten(), fisher=False))
        # bk_clamped = np.clip(bk, 2.0, 800.0)

        # p = float(np.interp(
        #     bk_clamped,
        #     [   3.0,   10.0,   30.0,  50.0,  100.0,  400.0,  800.0 ],
        #     [  97.0,   94.0,   88.0,  75.0,   65.0,   60.0,   55.0 ]
        # ))

        # # 3) compute the p-th percentile of the fused‐map
        # flat = fused.ravel()
        # threshold = float(np.percentile(flat, p))

        # 4) run your usual box generator
        boxes = generate_yolo_boxes(fused, threshold=0.04)

        # 2) run patch‐based pre‐filter + detection
        filtered = []
        H, W = block.shape
        for xc, yc, w_n, h_n in boxes:
            # map back to pixel coords
            x0 = max(0, int((xc - w_n/2) * W))
            x1 = min(W-1, int((xc + w_n/2) * W))
            y0 = max(0, int((yc - h_n/2) * H))
            y1 = min(H-1, int((yc + h_n/2) * H))

            patch = block[y0:y1, x0:x1].ravel()
            k_loc = kurtosis(patch, fisher=False) if patch.size >= 2 else 0.0

            if k_loc >= 4.5:         # only keep boxes over peaky patches
                filtered.append((xc, yc, w_n, h_n))

        # 3) continue with your existing scoring/NMS on boxes
        if len(filtered) == 0:
            boxes = []
        else:
            areas = [w*h for (_,_,w,h) in boxes]
            H, W = fused.shape
            kurt_vals = []
            for (cx, cy, w_n, h_n) in boxes:
                # convert normalized coords back to pixel indices
                x0 = max(0, int((cx - w_n/2) * W))
                x1 = min(W, int((cx + w_n/2) * W))
                y0 = max(0, int((cy - h_n/2) * H))
                y1 = min(H, int((cy + h_n/2) * H))

                patch = block[y0:y1, x0:x1].flatten()
                if patch.size < 2:
                    kurt_vals.append(0.0)
                else:
                    # use excess‐kurtosis (Gaussian = 0)
                    kurt_vals.append(kurtosis(patch, fisher=True))
                    # 4) Convert to numpy arrays
            a_arr = np.array(areas, dtype=float)
            k_arr = np.array(kurt_vals, dtype=float)

            # 5) Normalize safely
            a_norm = (a_arr - a_arr.min()) / (np.ptp(a_arr) + 1e-6)
            k_norm = (k_arr - k_arr.min()) / (np.ptp(k_arr) + 1e-6)

            # 6) Composite score & NMS
            scores = (0.9 * k_norm + 0.1 * a_norm).tolist()
            assert len(scores) == len(boxes), f"scores({len(scores)})!=boxes({len(boxes)})"
            boxes = non_max_suppression(boxes, scores, iou_thresh=0.3)
            boxes = shrink_large_boxes(
                boxes,
                fused,
                w_max=0.8,      # only shrink boxes ≥80% wide
                h_max=0.8,      # or ≥80% tall
                global_thresh=0.2   # threshold inside each patch
            )

            boxes, scores = remove_nested_boxes(boxes, scores)

        moe_boxes = boxes

        # 5) Convert all sources to raw pixel-space boxes
        H, W = fused.shape

        # a) MoE raw
        moe_raw = []
        for idx, (xc, yc, w_n, h_n) in enumerate(moe_boxes):
            x0 = max(0, int((xc - w_n/2) * W))
            x1 = min(W, int((xc + w_n/2) * W))
            # carry original MoE confidence (if available) or use 1.0
            moe_raw.append((x0, x1, { 'peak_snr': 1.0 }))
        #from bounds import threshold_baseline_bounds

        min_width   = 1
        max_width   = W
        snr_cutoff  = 7.0    # SNR threshold

        raw_refined = []
        for cx, cy, w_n, h_n in moe_boxes:
            # 1) back-project from normalized back to pixel coords
            l0 = max(0, int((cx - w_n/2) * W))
            r0 = min(W, int((cx + w_n/2) * W))
            if r0 - l0 < 2:
                # too narrow to do any threshold search
                continue

            spec = block.mean(axis=0)[l0:r0]
            if spec.size < 2:
                # absolutely no data
                continue

            # 2) compute local threshold offsets
            try:
                l_ref, r_ref, meta = threshold_baseline_bounds_numpy(spec, p=0.02)
            except (ValueError, IndexError):
                # threshold routine couldn’t find a crossing
                continue

            # 3) clamp offsets to valid spec-indices
            l_ref = int(np.clip(l_ref, 0, spec.size-2))
            r_ref = int(np.clip(r_ref, 1, spec.size-1))
            if r_ref <= l_ref:
                continue

            # 4) map back to global pixel coords
            new_l = l0 + l_ref
            new_r = l0 + r_ref
            width = new_r - new_l

            # 5) compute SNR and apply your filters
            snr_estimate = (spec.max() - meta['noise_mean']) / (meta['spec_max'] - meta['noise_mean'])
            if not (min_width < width < max_width and snr_estimate > snr_cutoff):
                continue

            # 6) append as raw pixel‐coords so fuse() can consume it
            raw_refined.append((new_l, new_r, meta))


        # c) Fallback raw (vertical + horizontal sweeps)
        #from bounds import clipped_2D_bounds
        stride, fwin = 50, 128
        meb, mcb, ppm = 3, 1, 4
        raw_fallback = []
        # vertical
        for start in range(0, W, stride):
            sub_block = block[:, start:start + fwin]
            if sub_block.shape[1] < 2:
                continue
            sub = sub_block.copy()
            try:
                l_rel, r_rel, meta = clipped_2D_bounds_numpy(sub, min_empty_bins=meb,
                                                    min_clipped=mcb,
                                                    peak_prominence=ppm)
            except (ValueError, IndexError):
                continue
            gl, gr = start + l_rel, start + r_rel
            w_pix = gr - gl
            if not (min_width < w_pix < max_width): continue
            raw_fallback.append((gl, gr, meta))
        # horizontal
        for t0 in range(0, H, stride):
            sub_block = block[t0:t0 + fwin, :]
            if sub_block.shape[0] < 2:
                continue
            sub = sub_block.copy()
            try:
                l_rel, r_rel, meta = clipped_2D_bounds_numpy(sub, min_empty_bins=meb,
                                                    min_clipped=mcb,
                                                    peak_prominence=ppm)
            except (ValueError, IndexError):
                continue
            gt, gb = t0 + l_rel, t0 + r_rel
            h_pix = gb - gt
            if not (min_width < h_pix < max_width): continue
            # tag horizontal boxes with top/bottom
            meta_h = meta.copy(); meta_h.update({'top': gt, 'bottom': gb})
            raw_fallback.append((0, W, meta_h))

            # 6) Fuse all raw boxes (pixel-space)
        all_raw = [moe_raw, raw_refined, raw_fallback]
        fused_raw = fuse(all_raw, iou_thresh=0.5)

        # 7) Normalize fused boxes back to YOLO format
        yolo_boxes = []  # (class_id, xc, yc, w, h)
        for l, r, meta in fused_raw:
            if 'top' in meta:
                x0, x1 = 0, W
                y0, y1 = meta['top'], meta['bottom']
            else:
                x0, x1 = l, r
                y0, y1 = 0, H
            w_pix, h_pix = x1 - x0, y1 - y0
            cx = x0 + w_pix/2; cy = y0 + h_pix/2
            yolo_boxes.append((class_id,
                                cx / W,
                                cy / H,
                                w_pix / W,
                                h_pix / H))


        f_start = fb.header['fch1'] + f0 * fb.header['foff']
        f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']
        kurt_val = kurtosis(gray8.flatten(), fisher=False)
        fn = f"img_{kurt_val:0{pad_width}.2f}_f_{f_start:.4f}_{f_stop:.4f}.png"
        img_path = img_dir / fn
        txt_path = lbl_dir / fn.replace(".png", ".txt")

        arr8 = (255 * (block - block.min()) / (np.ptp(block) + 1e-6)).astype(np.uint8)
        img = Image.fromarray(np.stack([arr8]*3, axis=-1))
        img.save(img_path)

        full_image_threshold = 0.95
        min_norm_area        = 0.005
        # boxes is List[(x_c, y_c, w_n, h_n)]
        boxes = [
            (_, x_c, y_c, w_n, h_n)
            for _, x_c, y_c, w_n, h_n in yolo_boxes
            if min_norm_area <= (w_n * h_n) <= full_image_threshold
        ]

        with open(txt_path, "w") as f:
            for _, x_c, y_c, w_n, h_n in boxes:
                f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}\n")


        if len(boxes) == 0:
            (lbl_dir/"empty_labels.csv").open("a").write(f"{fn},{h5_path},{ch_idx}\n")

        vis_img = img.convert("RGB")
        draw    = ImageDraw.Draw(vis_img)
        for _, x_c, y_c, w_n, h_n in boxes:
            xc, yc = x_c*w_img, y_c*h_img
            bw, bh = w_n*w_img, h_n*h_img
            x0, y0 = int(xc - bw/2), int(yc - bh/2)
            x1, y1 = int(xc + bw/2), int(yc + bh/2)
            draw.rectangle([x0, y0, x1, y1], outline="red", width=2)
        vis_img.save(vis_dir/fn)

if __name__ == "__main__":
    tasks = build_tasks(df)
    train_set, val_set = split_tasks(tasks)

    # 2) build job tuples of exactly what process_file unpacks:
    job_args = []
    BASE_DIR = "/datax/scratch/jliang/dataset_moe_bounds-gate"
    NUM_WORKERS = 8
    class_id = 0  # Assuming a single class for simplicity
    pad_width = 4  # Zero-padding width for gidx in filenames
    for idx, (h5_path, ch_idx) in enumerate(tasks):
        job = (
            h5_path,
            [ch_idx],        # channels
            [idx],           # gidxs
            BASE_DIR,
            class_id,
            train_set,
            val_set,
            pad_width
        )
        job_args.append(job)

    # 3) submit each job as a single argument
    with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
        futures = {
            exe.submit(process_file, job): (job[0], job[1])
            for job in job_args
        }

        for future in tqdm(as_completed(futures), total=len(futures)):
            h5, ch_list = futures[future]
            try:
                future.result()
            except Exception as e:
                print(f"Error processing {h5} channel {ch_list}: {e}")
                traceback.print_exc()


In [ ]:
# import numpy as np
# import cv2
# import torch
# import torch.nn as nn
# from skimage.segmentation import slic
# from scipy import ndimage

# # ─── 1) HEURISTIC COMPUTATION ─────────────────────────────────────────

# def compute_heuristics(edge_map, gray_image):
#     # edge_map: 2D float32 [0,1], gray_image: 2D uint8
#     # 1) gradient magnitude
#     gx = cv2.Sobel(gray_image, cv2.CV_32F, 1, 0, ksize=3)
#     gy = cv2.Sobel(gray_image, cv2.CV_32F, 0, 1, ksize=3)
#     grad = np.hypot(gx, gy)

#     # 2) statistics
#     density   = np.mean(edge_map > 0.2)
#     align     = np.mean(grad[edge_map > 0.2]) if density>0 else 0.0
#     entropy   = -np.sum(edge_map * np.log2(edge_map + 1e-8))
#     # 3) linearity via eccentricity of components
#     bw   = edge_map > 0.2
#     lbl, n = ndimage.label(bw)
#     props = ndimage.find_objects(lbl)
#     eccs = []
#     for i, sl in enumerate(props, start=1):
#         region = (lbl[sl] == i).astype(np.uint8)
#         # approximate by second moments → skip details here
#         eccs.append(region.sum()>0 and region.sum() / (region.shape[0]*region.shape[1]))
#     linearity = float(np.mean(eccs)) if eccs else 0.0

#     if density < 0.01:
#         return [0.0, 0.0, 0.0, 0.0]
#     return [density, align, entropy, linearity]


# # ─── 2) RULE-BASED GATE MODULE ──────────────────────────────────────────

# class RuleGate(nn.Module):
#     def __init__(self, w_align=1.0, w_ent=1.0, bias=0.0):
#         super().__init__()
#         # wrap as parameters if you want to tune via backprop:
#         self.w_align = nn.Parameter(torch.tensor(w_align))
#         self.w_ent   = nn.Parameter(torch.tensor(w_ent))
#         self.bias    = nn.Parameter(torch.tensor(bias))

#     def forward(self, h_sf, h_ued):
#         # h_*: [B,4] tensors on CUDA
#         align = h_ued[:,1] - h_sf[:,1]      # favors UED if >0
#         ent   = h_sf[:,2] - h_ued[:,2]      # favors UED if >0
#         score  = self.w_align * align + self.w_ent * ent + self.bias
#         return torch.sigmoid(score)         # [B] weights for UED


# class MoEBlock(nn.Module):
#     def __init__(self, gate: RuleGate):
#         super().__init__()
#         self.gate = gate

#     def forward(self, sf_map, ued_map, h_sf, h_ued):
#         """
#         sf_map, ued_map: [B,H,W] floats on CUDA
#         h_sf, h_ued:        [B,4] heuristic tensors on CUDA
#         """
#         p = self.gate(h_sf, h_ued).view(-1,1,1)  # [B,1,1]
#         return p * ued_map + (1 - p) * sf_map


# # ─── 3) FULL PROCESSING LOOP ───────────────────────────────────────────

# def process_image_patch(patch, sf_detector, gate_block, device='cuda'):
#     """
#     patch: HxWx3 BGR uint8
#     sf_detector: cv2.ximgproc StructuredEdgeDetection
#     gate_block:   instance of MoEBlock on device
#     """
#     # 1) compute both edge maps
#     gray    = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
#     sf_edges= sf_detector.detectEdges(np.float32(patch)/255.0).squeeze()
#     gx = cv2.Sobel(gray, cv2.CV_32F, 1,0,3)
#     gy = cv2.Sobel(gray, cv2.CV_32F, 0,1,3)
#     ued_edges = np.hypot(gx, gy)
#     ued_edges = (ued_edges/ued_edges.max()).astype(np.float32)

#     # 2) superpixel segmentation
#     segments = slic(patch, n_segments=100, compactness=10)

#     # 3) gather heuristics and masks
#     feats_sf, feats_ued, masks = [], [], []
#     for seg in np.unique(segments):
#         mask = (segments==seg)
#         if mask.sum()<50: continue
#         hsf = compute_heuristics(sf_edges*mask, gray)
#         hud = compute_heuristics(ued_edges*mask, gray)
#         if not any(hsf) and not any(hud): continue
#         feats_sf.append(hsf)
#         feats_ued.append(hud)
#         masks.append(mask)

#     if not masks:
#         return np.zeros_like(sf_edges, np.float32)

#     # 4) move to GPU
#     h_sf = torch.tensor(feats_sf, device=device)
#     h_ued= torch.tensor(feats_ued, device=device)
#     sf_m = torch.tensor(np.stack([sf_edges[m] for m in masks]), device=device)
#     ue_m = torch.tensor(np.stack([ued_edges[m] for m in masks]), device=device)

#     # 5) fuse with gate
#     fused_masks = gate_block(sf_m, ue_m, h_sf, h_ued)  # [N_pixels]
#     fused = np.zeros_like(sf_edges, np.float32)
#     for p, m in zip(fused_masks.cpu().numpy(), masks):
#         fused[m] = p

#     # 6) generate YOLO boxes
#     return fused


# # ─── 4) USAGE ─────────────────────────────────────────────────────────

# # Initialize:
# EDGE_MODEL_PATH = "model.yml"  # your SF model
# sf_detector = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL_PATH)
# gate        = RuleGate(w_align=2.0, w_ent=1.0, bias=-0.5).cuda()
# moe_block   = MoEBlock(gate).cuda()

# # For each patch:
# # fused_map = process_image_patch(patch, sf_detector, moe_block)
# # then threshold fused_map, label, convert to YOLO (cx,cy,w,h) as before.


In [ ]:
# def compute_heuristics(image, edge_map):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     gradient = sobel(gray)
#     edge_density = np.mean(edge_map > 0.2)
#     edge_gradient_alignment = np.mean(gradient[edge_map > 0.2])
#     edge_entropy = -np.sum(edge_map * np.log2(edge_map + 1e-8))
#     lin = linearity_score(edge_map)
#     if edge_density < 0.01:
#         return [0.0, 0.0, 0.0, 0.0]
#     return [edge_density, edge_gradient_alignment, edge_entropy, lin]

# def pseudo_labels(image, sf_edges, ued_edges):
#     h_sf  = compute_heuristics(image, sf_edges)
#     h_ued = compute_heuristics(image, ued_edges)
#     # if UED better alignment & lower entropy, choose UED
#     if h_ued[1] > h_sf[1] and h_ued[2] < h_sf[2]:
#         return 0
#     else:
#         return 1

# # PyTorch gating network (input_dim=8 for 4 heuristics each)
# class GatingNet(nn.Module):
#     def __init__(self, input_dim=8, hidden=16):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(input_dim, hidden),
#             nn.ReLU(),
#             nn.Linear(hidden, 2)
#         )
#     def forward(self, x):
#         return self.net(x)

# # Train gating model in PyTorch
# def train_gating_model_torch(images, sf_edge_maps, ued_edge_maps,
#                              batch_size=64, epochs=10, lr=1e-3,
#                              device='cuda'):
#     # 1) Collect features & labels
#     feats, labels = [], []
#     for img, sf, ued in zip(images, sf_edge_maps, ued_edge_maps):
#         segments = slic(img_as_float(img), n_segments=100, compactness=10)
#         for seg_val in np.unique(segments):
#             mask = (segments == seg_val)
#             if mask.sum() < 50:
#                 continue
#             sf_patch, ued_patch = sf * mask, ued * mask
#             if sf_patch.mean() + ued_patch.mean() < 0.01:
#                 continue
#             h_sf  = compute_heuristics(img, sf_patch)
#             h_ued = compute_heuristics(img, ued_patch)
#             feats.append(np.array(h_sf + h_ued, dtype=np.float32))
#             labels.append(pseudo_labels(img, sf_patch, ued_patch))
#     if len(set(labels)) < 2:
#         return None, None, None

#     # 2) Build tensors & standardize
#     X = torch.tensor(feats)  # (N, 8)
#     y = torch.tensor(labels, dtype=torch.long)  # (N,)
#     mean, std = X.mean(dim=0, keepdim=True), X.std(dim=0, keepdim=True) + 1e-6
#     X = (X - mean) / std

#     # 3) DataLoader
#     dataset = TensorDataset(X, y)
#     loader  = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#     # 4) Model, loss, optimizer
#     model = GatingNet(input_dim=X.shape[1]).to(device)
#     optimizer = optim.Adam(model.parameters(), lr=lr)
#     criterion = nn.CrossEntropyLoss()

#     # 5) Training loop
#     model.train()
#     for epoch in range(epochs):
#         total_loss = 0.0
#         for xb, yb in loader:
#             xb, yb = xb.to(device), yb.to(device)
#             logits = model(xb)
#             loss   = criterion(logits, yb)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
#         print(f"Epoch {epoch+1}/{epochs} - loss: {total_loss/len(loader):.4f}")

#     return model, mean.to(device), std.to(device)

# # Apply gating model to fuse edges
# def apply_gating_model_torch(image, sf_edges, ued_edges,
#                              model, mean, std, device='cuda'):
#     segments = slic(img_as_float(image), n_segments=100, compactness=10)
#     feats, masks = [], []
#     for seg_val in np.unique(segments):
#         mask = (segments == seg_val)
#         if mask.sum() < 50:
#             continue
#         sf_patch, ued_patch = sf_edges * mask, ued_edges * mask
#         if sf_patch.mean() + ued_patch.mean() < 0.01:
#             continue
#         h_sf, h_ued = compute_heuristics(image, sf_patch), compute_heuristics(image, ued_patch)
#         f = np.array(h_sf + h_ued, dtype=np.float32)
#         if not np.all(np.isfinite(f)) or np.allclose(f, 0, atol=1e-3):
#             continue
#         feats.append(f)
#         masks.append(mask)

#     if not feats:
#         return np.zeros_like(sf_edges, dtype=float)

#     X = torch.tensor(feats).to(device)
#     X = (X - mean) / std
#     model.eval()
#     with torch.no_grad():
#         probs = torch.softmax(model(X), dim=1)[:,1].cpu().numpy()

#     gated = np.zeros_like(sf_edges, dtype=float)
#     for p, mask in zip(probs, masks):
#         gated[mask] = p * sf_edges[mask] + (1 - p) * ued_edges[mask]
#     return gated


In [ ]:
# def edge_detection(gray, factor=6, dilation=(3, 3)):
#         # STEP 1: apply power threshold to suppress background
#         median = np.median(gray)
#         mad = np.median(np.abs(gray - median))
#         power_mask = gray > (median + factor * mad)

#         # STEP 2: edge detection on filtered signal only
#         grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
#         grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
#         magnitude = np.sqrt(grad_x**2 + grad_y**2)
#         magnitude *= power_mask  # mask out noise
#         magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

#         # STEP 3: binary threshold + dilation
#         binary = magnitude > 50  # you can tune this
#         binary = scipy.ndimage.binary_dilation(binary, structure=np.ones(dilation))
#         labeled, n_objs = scipy.ndimage.label(binary)
#         slices = scipy.ndimage.find_objects(labeled)
#         return slices

# EDGE_MODEL_PATH = '/home/jliang/gbt-rfi/model.yml.gz'
# EDGE_DETECTOR = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL_PATH)

# def detect_edges(spectrogram):
#     img = (spectrogram - spectrogram.min()) / (spectrogram.ptp() + 1e-6)
#     img_3ch = cv2.merge([img.astype(np.float32)] * 3)
#     return EDGE_DETECTOR.detectEdges(img_3ch)

In [ ]:
# def extract_one_sample(h5_path, ch, edge_model_path='/home/jliang/gbt-rfi/model.yml.gz'):
#     try:
#         EDGE_DETECTOR = cv2.ximgproc.createStructuredEdgeDetection(edge_model_path)
#         fb = bl.Waterfall(h5_path, load_data=True)
#         data = 10 * np.log10(fb.data.squeeze())

#         nfpc = fb.header.get("nfpc", 1024)
#         f0, f1 = ch * nfpc, (ch + 1) * nfpc
#         block = data[:, f0:f1]

#         n_cols = block.shape[1]
#         low, high = int(0.1 * n_cols), int(0.9 * n_cols)
#         block_middle80 = block[:, low:high]

#         vert_means = block_middle80.mean(axis=0)
#         center = np.argmax(vert_means)
#         left_col, right_col = center - 1, center + 1
#         if left_col >= 0 and right_col < block_middle80.shape[1]:
#             block_middle80[:, center] = (block_middle80[:, left_col] + block_middle80[:, right_col]) / 2
#         elif left_col >= 0:
#             block_middle80[:, center] = block_middle80[:, left_col]
#         elif right_col < block_middle80.shape[1]:
#             block_middle80[:, center] = block_middle80[:, right_col]

#         img_norm = (block_middle80 - block_middle80.min()) / (block_middle80.ptp() + 1e-6)
#         img_rgb = cv2.merge([img_norm.astype(np.float32)] * 3)

#         sf_edges = EDGE_DETECTOR.detectEdges(img_rgb)

#         gray = (255 * img_norm).astype(np.uint8)
#         grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
#         grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
#         magnitude = np.sqrt(grad_x**2 + grad_y**2)
#         magnitude = cv2.normalize(magnitude, None, 0, 1, cv2.NORM_MINMAX)

#         sf_mean = np.mean(sf_edges)
#         ued_mean = np.mean(magnitude)

#         if sf_mean + ued_mean < 0.005:  # threshold to skip empty or background-only blocks
#             return None  # skip empty or background-only blocks


#         return (img_rgb, sf_edges, magnitude)

#     except Exception as e:
#         print(f"Error processing {h5_path} ch {ch}: {e}")
#         return None

# def extract_samples_batched(df, total_samples=40000, batch_size=32, num_workers=4):
#     all_tasks = []
#     for _, row in df.iterrows():
#         h5 = row[".h5 path"]
#         fb = bl.Waterfall(h5, load_data=False)
#         nfreq = fb.header.get("nchans")
#         nfpc = fb.header.get("nfpc", 1024)
#         for ch in range(nfreq // nfpc):
#             all_tasks.append((h5, ch))

#     shuffle(all_tasks)
#     selected_tasks = all_tasks[:total_samples]

#     train_images, train_sf_edges, train_ued_edges = [], [], []
#     for i in tqdm(range(0, total_samples, batch_size)):
#         batch = selected_tasks[i:i+batch_size]
#         with ProcessPoolExecutor(max_workers=num_workers) as executor:
#             futures = [executor.submit(extract_one_sample, h5, ch) for h5, ch in batch]
#             for future in as_completed(futures):
#                 result = future.result()
#                 if result:
#                     img_rgb, sf_edge, ued_edge = result
#                     train_images.append(img_rgb)
#                     train_sf_edges.append(sf_edge)
#                     train_ued_edges.append(ued_edge)

#     return train_images, train_sf_edges, train_ued_edges



In [ ]:
# # 4) SET UP LOGGING
# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     datefmt="%Y-%m-%d %H:%M:%S",
# )

# from gating_model import train_gating_model_torch

# def build_tasks(df):
#     """
#     Build a flat list of (h5_path, channel_idx) from your DataFrame,
#     skipping any channels that fall into known notch filter frequency ranges.
#     """
#     GBT_NOTCH_FILTERS = {
#         "L": [(1200, 1340)],
#         "S": [(2300, 2360)],
#     }

#     tasks = []

#     for _, row in df.iterrows():
#         h5 = row[".h5 path"]
#         band = row["Band"]  # e.g., 'L', 'S', etc.
#         fb = bl.Waterfall(h5, load_data=False)
#         nfreq = fb.header.get("nchans")
#         nfpc = fb.header.get("nfpc", 1024)
#         fch1 = fb.header["fch1"]
#         foff = fb.header["foff"]
#         n_coarse = nfreq // nfpc

#         for ch in range(n_coarse):
#             f0 = fch1 + ch * nfpc * foff
#             f1 = fch1 + (ch + 1) * nfpc * foff
#             f_min, f_max = sorted([f0, f1])

#             # Check against notch filter exclusion ranges
#             skip = False
#             if band in GBT_NOTCH_FILTERS:
#                 for lo, hi in GBT_NOTCH_FILTERS[band]:
#                     if lo <= f_min <= hi or lo <= f_max <= hi:
#                         skip = True
#                         break
#             if not skip:
#                 tasks.append((h5, ch))

#     return tasks


# def split_tasks(tasks, train_frac=0.8, seed=42):
#     """
#     Shuffle & split the flat task list into train vs. val sets.
#     Returns two sets of (h5_path, channel_idx).
#     """
#     random.seed(seed)
#     shuffled = tasks.copy()
#     random.shuffle(shuffled)
#     cut = int(train_frac * len(shuffled))
#     train = set(shuffled[:cut])
#     val   = set(shuffled[cut:])
#     return train, val

# def process_file(job):
#     """
#     job is a tuple:
#       (h5_path, channels, global_indices,
#        base_dir, factor, dilation,
#        class_id, train_set, val_set,
#        pad_width, model, mean, std)
#     """
#     (h5_path, channels, global_indices,
#      base_dir, factor, dilation,
#      class_id, train_set, val_set,
#      pad_width, model, mean, std) = job

#     # 1) Prepare device & trained gate
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)
#     model.eval()

#     # 2) Load Structured Forest detector once
#     EDGE_MODEL_PATH = "/home/jliang/gbt-rfi/model.yml.gz"
#     sf_detector = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL_PATH)

#     # 3) Load the .h5 file
#     fb   = bl.Waterfall(h5_path, load_data=True)
#     data = 10 * np.log10(fb.data.squeeze())   # (ntime, nfreq)

#     # 4) Process each coarse channel
#     for ch_idx, gidx in zip(channels, global_indices):
#         # decide train vs val
#         subset = "train" if (h5_path, ch_idx) in train_set else "val"

#         # make sure directories exist
#         img_dir = Path(base_dir)/subset/"images"
#         lbl_dir = Path(base_dir)/subset/"labels"
#         vis_dir = Path(base_dir)/"visualization"/subset
#         for d in (img_dir, lbl_dir, vis_dir):
#             d.mkdir(parents=True, exist_ok=True)

#         # extract the 80% middle of the block
#         cw = fb.header.get("nfpc", 1024)
#         f0, f1 = ch_idx*cw, (ch_idx+1)*cw
#         block = data[:, f0:f1]
#         low, high = int(0.1*cw), int(0.9*cw)
#         block = block[:, low:high]

#         # remove single‐column artifact
#         col_means = block.mean(axis=0)
#         c = int(np.argmax(col_means))
#         if 0 < c < block.shape[1]-1:
#             block[:,c] = 0.5*(block[:,c-1] + block[:,c+1])

#         # normalize & build RGB input for SF
#         norm = (block - block.min())/(block.ptp()+1e-6)
#         img_rgb = cv2.merge([norm.astype(np.float32)]*3)

#         # compute the two expert edge‐maps
#         sf_edge_map  = sf_detector.detectEdges(img_rgb).squeeze()
#         gray_uint8  = (255*norm).astype(np.uint8)
#         gx = cv2.Sobel(gray_uint8, cv2.CV_32F, 1, 0, ksize=3)
#         gy = cv2.Sobel(gray_uint8, cv2.CV_32F, 0, 1, ksize=3)
#         ued_edge_map = np.hypot(gx, gy)
#         ued_edge_map = cv2.normalize(ued_edge_map, None, 0.0, 1.0, cv2.NORM_MINMAX)

#         # 5) Fuse using the PyTorch gate
#         fused_edges = apply_gating_model_torch(
#             img_rgb, sf_edge_map, ued_edge_map,
#             model, mean, std,
#             device=device
#         )

#         # 6) Generate & save YOLO boxes
#         boxes = generate_yolo_boxes(fused_edges, threshold=0.1)

#         # Dimensions for filtering
#         h_img, w_img = block.shape
#         min_size = 3                   # px
#         full_image_threshold = 0.95    # normalized area

#         # Filter
#         final_boxes = []
#         for x_c, y_c, w_n, h_n, area in boxes:
#             if area > full_image_threshold:
#                 continue
#             if (w_n * w_img) < min_size or (h_n * h_img) < min_size:
#                 continue
#             final_boxes.append((x_c, y_c, w_n, h_n))

#         # Calculate frequency range of this coarse channel
#         f_start = fb.header['fch1'] + f0 * fb.header['foff']
#         f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']

#         # Build filename
#         fn = f"img_{gidx:0{pad_width}d}_f_{f_start:.4f}_{f_stop:.4f}.png"
#         img_path = img_dir / fn
#         txt_path = lbl_dir / fn.replace(".png", ".txt")

#         # Save image
#         arr8 = (255 * (block - block.min()) / (block.ptp() + 1e-6)).astype(np.uint8)
#         img = Image.fromarray(np.stack([arr8]*3, axis=-1))
#         img.save(img_path)

#         # Write YOLO labels
#         with open(txt_path, "w") as f:
#             for x_c, y_c, w_n, h_n in final_boxes:
#                 f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}\n")

#         # If empty, log it
#         if not final_boxes:
#             (lbl_dir/"empty_labels.csv").open("a").write(f"{fn},{h5_path},{ch_idx}\n")

#         # Draw & save visualization
#         vis_img = img.convert("RGB")
#         draw    = ImageDraw.Draw(vis_img)
#         for x_c, y_c, w_n, h_n in final_boxes:
#             xc, yc = x_c*w_img, y_c*h_img
#             bw, bh = w_n*w_img, h_n*h_img
#             x0, y0 = int(xc - bw/2), int(yc - bh/2)
#             x1, y1 = int(xc + bw/2), int(yc + bh/2)
#             draw.rectangle([x0, y0, x1, y1], outline="red", width=2)
#         vis_img.save(vis_dir/fn)



# if __name__ == "__main__":
#     # === your settings ===
#     BASE_DIR             = "/datax/scratch/jliang/dataset_moe"
#     FACTOR              = 6
#     DILATION = (30, 30)
#     CLASS_ID             = 0
#     TRAIN_FRAC           = 0.8
#     SEED                 = 42
#     NUM_WORKERS          = 2

#     # --- assume you already have a DataFrame `df` with at least 'h5_path' and optionally 'nchans' ---
#     # df = pd.read_csv(...)  # or however you built it

#     # 1) Build & split tasks
#     tasks = build_tasks(df)
#     train_set, val_set = split_tasks(tasks, TRAIN_FRAC, SEED)

#     # 2) compute zero-pad width from total images
#     pad_width = len(str(len(tasks) - 1))

#     # 3) regroup tasks by file to load each .h5 only once
#     jobs = {}
#     for gidx, (h5, ch) in enumerate(tasks):
#         jobs.setdefault(h5, {"chs": [], "gidxs": []})
#         jobs[h5]["chs"].append(ch)
#         jobs[h5]["gidxs"].append(gidx)

#     # 4) prepare job‐tuples
#     job_list = []
#     for h5, info in jobs.items():
#         job_list.append((
#             h5,
#             info["chs"],
#             info["gidxs"],
#             BASE_DIR,
#             FACTOR,
#             DILATION,
#             CLASS_ID,
#             train_set,
#             val_set,
#             pad_width
#         ))

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     # train_images, train_sf_edges, train_ued_edges = extract_samples_batched(
#     #     df, total_samples=40000, num_workers=2
#     # )

#     # model, mean, std = train_gating_model_torch(
#     #     train_images,
#     #     train_sf_edges,
#     #     train_ued_edges,
#     #     batch_size=64,
#     #     epochs=10,
#     #     lr=1e-3,
#     #     device=device
#     # )

#     # if model is None:
#     #     logging.warning("Skipping inference because gate net couldn't train.")
#     #     exit()


In [ ]:
# import h5py, numpy as np, torch, logging
# from torch.utils.data import IterableDataset, DataLoader
# import cv2
# from skimage.segmentation import slic
# from yolo_moe_pytorch_channel import (
#     apply_gating_model_torch,
#     generate_yolo_boxes,
#     compute_heuristics,
#     pseudo_labels
# )
# from PIL import Image, ImageDraw

# # 1) Streaming dataset
# class H5ChannelDataset(IterableDataset):
#     def __init__(self, df, nfpc=1024):
#         self.paths = df[".h5 path"].tolist()
#         self.nfpc  = nfpc
#     def __iter__(self):
#         for p in self.paths:
#             with h5py.File(p, "r") as f:
#                 d = f["data"]
#                 n_coarse = d.shape[1] // self.nfpc
#                 for ch in range(n_coarse):
#                     yield d[:, ch*self.nfpc:(ch+1)*self.nfpc].astype(np.float32)

# # 2) Pre- and post-processing helpers
# EDGE_MODEL_PATH = "/home/jliang/gbt-rfi/model.yml.gz"
# sf_detector    = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL_PATH)

# def preprocess(block):
#     norm = (block - block.min())/(block.ptp()+1e-6)
#     img_rgb = np.stack([norm]*3, axis=-1).astype(np.float32)
#     sf_map   = sf_detector.detectEdges(img_rgb).squeeze()
#     gray8    = (255*norm).astype(np.uint8)
#     gx = cv2.Sobel(gray8, cv2.CV_32F, 1,0,3)
#     gy = cv2.Sobel(gray8, cv2.CV_32F, 0,1,3)
#     ued_map = np.hypot(gx, gy)
#     ued_map = cv2.normalize(ued_map, None, 0.0, 1.0, cv2.NORM_MINMAX)
#     return img_rgb, sf_map, ued_map

# def segment_patches(img, sf, ued):
#     segs = slic(img, n_segments=100, compactness=10)
#     for v in np.unique(segs):
#         m = segs==v
#         if m.sum()<50: continue
#         sf_p, ued_p = sf*m, ued*m
#         if sf_p.mean()+ued_p.mean()<0.01: continue
#         yield sf_p, ued_p

# # 3) Collate: build feature / label tensors
# def collate_fn(batch):
#     feats, labs = [], []
#     for block in batch:
#         img, sf, ued = preprocess(block)
#         for sf_p, ued_p in segment_patches(img, sf, ued):
#             h_sf = compute_heuristics(sf_p, img)
#             h_ued= compute_heuristics(ued_p, img)
#             feats.append(h_sf + h_ued)
#             labs.append(pseudo_labels(img, sf_p, ued_p))
#     if not feats: return None
#     return torch.tensor(feats), torch.tensor(labs)

# # 4) Training
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# dataset = H5ChannelDataset(df)
# loader  = DataLoader(dataset,
#                      batch_size=4,
#                      num_workers=4,
#                      persistent_workers=True,
#                      collate_fn=collate_fn)

# from your_module import GatingNet  # the BatchNorm version
# net   = GatingNet().to(device)
# opt   = torch.optim.Adam(net.parameters(), lr=1e-3)
# crit  = nn.CrossEntropyLoss()

# for epoch in range(10):
#     for batch in loader:
#         if batch is None: continue
#         X, y = batch
#         X, y = X.to(device), y.to(device)
#         logits = net(X)
#         loss   = crit(logits, y)
#         opt.zero_grad()
#         loss.backward()
#         opt.step()
#     print(f"Epoch {epoch+1}/10 done")

# # 5) Build job_list as before, but append (net) instead of (mean,std)
# # and in process_file call apply_gating_model_torch(img, sf, ued, net, device=...)

# # 6) Inference uses your existing process_file, no further change needed.


# # ─── 3) Build job list for inference ────────────────────────────────────

# # your existing build_tasks(), split_tasks(), jobs grouping...
# tasks    = build_tasks(df)
# train_set, val_set = split_tasks(tasks, TRAIN_FRAC, SEED)
# pad_width = len(str(len(tasks)))

# jobs = {}
# for gidx,(h5,ch) in enumerate(tasks):
#     jobs.setdefault(h5, {"chs":[], "gidxs":[]})
#     jobs[h5]["chs"].append(ch)
#     jobs[h5]["gidxs"].append(gidx)

# job_list = []
# for h5,info in jobs.items():
#     job_list.append((h5,
#                      info["chs"],
#                      info["gidxs"],
#                      BASE_DIR,
#                      FACTOR,
#                      DILATION,
#                      CLASS_ID,
#                      train_set,
#                      val_set,
#                      pad_width,
#                      gate_net, mean, std))

# # ─── 4) Run inference in parallel ───────────────────────────────────────
# from concurrent.futures import ProcessPoolExecutor

# with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
#     for _ in exe.map(process_file, job_list):
#         pass
